In [1]:
import pandas as pd
import re
from tqdm.auto import tqdm
import yfinance as yf
import swifter
import numpy as np
from collections import OrderedDict

#載入PCA套件
from sklearn.decomposition import PCA

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tmp = pd.read_excel('Data/codes.xlsx', header = 1)
del tmp['#']
db = dict()
for i in range(0,len(tmp)):
    if tmp['英文代碼'][i] not in db.keys():
        db[tmp['英文代碼'][i]] = []
    db[tmp['英文代碼'][i]] =tmp['中文簡稱'][i]


In [3]:
df = pd.DataFrame()
for i in range(2011, 2023):
    try:
        df_ = pd.read_csv('Data/'+str(i)+'_fut.csv', encoding = 'cp950', index_col = 0, low_memory=False)[['契約', '到期月份(週別)', '開盤價', '最高價', '最低價', '收盤價', '漲跌價', '漲跌%', '成交量','結算價', '未沖銷契約數', '最後最佳買價', '最後最佳賣價', '歷史最高價', '歷史最低價']]
    except:
        print(i)
        df_ = pd.read_csv('Data/'+str(i)+'_fut.csv', low_memory=False , index_col = 0)
    df = pd.concat([df,df_])

2018
2019
2020
2021
2022


In [4]:
df['中文名稱'] = df['契約'].apply(lambda x: db[x])
df['Type'] = df['中文名稱'].apply(lambda x: 0 if x in ['非金電期貨', '臺股期貨', '臺幣黃金期貨', '金融期貨', '電子期貨', '臺灣50期貨', '小型臺指期貨', '櫃買期貨', '黃金期貨', '十年期公債期貨', '三十天期利率期貨', '摩臺期'] else 1)
df['Type_'] = df['到期月份(週別)'].apply(lambda x: 0 if '/' in str(x) else 1)
df = df[(df['Type'] == 1) & (df['Type_'] == 1) & (df.index != '')] 
df.index = pd.to_datetime(df.index) 

In [5]:
filter1 = re.compile(r'(.*?)[期]')
re.findall(filter1, '中信金期貨')
df['中文名稱'] = df['中文名稱'].apply(lambda x: re.findall(filter1, x)[0])
df['交易日期'] = df.index

In [6]:
TEJ = pd.read_excel('Data/TEJ_20231126.xlsx')

In [7]:
filter2 = re.compile(r'(.*?)[證]')
TEJ['名稱'] = TEJ['名稱'].apply(lambda x: re.findall(filter2, x)[0] if x[-2:] == '證券' else x)
TEJ['財報發布日'] = pd.to_datetime(TEJ['財報發布日']) 

In [8]:
def find_closest_date(target_date, dates):
    try:
        tmp = dates.iloc[(dates - target_date).abs().argsort()[:1]].values[0]
    except:
        tmp = 'None'
    return tmp

In [9]:
# df['財報發布日']  = df.apply(lambda x: find_closest_date(x['交易日期'],TEJ['財報發布日'][TEJ['名稱'] == x['中文名稱']]), axis = 1)


df['財報發布日']  = df.swifter.apply(lambda x: find_closest_date(x['交易日期'],TEJ['財報發布日'][TEJ['名稱'] == x['中文名稱']]), axis = 1)

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/swifter/swifter.py:88: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, "
Pandas Apply: 100%|██████████| 3356904/3356904 [3:04:47<00:00, 302.75it/s]  


In [10]:
df.to_csv('df_merge.csv', index =False)

In [11]:

# tmps = []
# error_tmps = []
# pbar = tqdm(df.iterrows(), total = (len(df)))
# for index, row in pbar:
#     try:
#         tmp = find_closest_date(row['交易日期'], TEJ['財報發布日'][TEJ['名稱'] == row['中文名稱']])
#         tmps.append(tmp)
#     except:
#         error_tmps.append(row['中文名稱'])

In [12]:
df['財報發布日'] = df['財報發布日'].apply(lambda x: np.nan if x == 'None' else x)
df['財報發布日'] = pd.to_datetime(df['財報發布日'])

# 直接從這裏跑

In [13]:
TEJ = pd.read_excel('Data/TEJ_20231126.xlsx')
filter2 = re.compile(r'(.*?)[證]')
TEJ['名稱'] = TEJ['名稱'].apply(lambda x: re.findall(filter2, x)[0] if x[-2:] == '證券' else x)
TEJ['財報發布日'] = pd.to_datetime(TEJ['財報發布日']) 

In [14]:
df = pd.read_csv('df_merge.csv')
df['交易時段'] = df['交易時段'].fillna('一般')
df = df[df['交易時段'] == '一般']
df['財報發布日'] = df['財報發布日'].apply(lambda x: np.nan if x == 'None' else x)
df['財報發布日'] = pd.to_datetime(df['財報發布日'])

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (8,9,10,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
data_code = pd.read_excel('Data/股名代碼.xlsx', header = 2)

In [16]:
data_code = data_code.T.reset_index(drop=True).T

In [17]:
tmp_1 = []
tmp_2 = []
tmp_3 = []
for i in range(0, len(data_code.T)):
    for j in range(0,len(data_code[i])):
        if pd.isna(data_code[i][j]) == False:
            if i%2 == 0:
                tmp_1.append(data_code[i][j].split('\xa0')[0])
            else:
                tmp_2.append(data_code[i][j].split('\xa0')[0])

df_code = pd.DataFrame()

In [18]:
df_code = pd.DataFrame()
df_code['code'] = tmp_1
df_code['name'] = tmp_2

tmp_3 = []
tmp_4 = []
market = ''
sector = '' 
for i in range(len(tmp_1)):
    if tmp_1[i] == '上市' or tmp_1[i] == '上櫃':
       market = tmp_1[i]
       sector = tmp_2[i]
    tmp_3.append(market)
    tmp_4.append(sector)

df_code['sector'] = tmp_4
df_code['market'] = tmp_3
df_code['name'] = df_code['name'].apply(lambda x: str(x.split("＊")[0]) if len(x.split("＊"))>0 else str(x))
df_code['name'] = df_code['name'].apply(lambda x: str(x.split("＃")[0]) if len(x.split("＃"))>0 else str(x))

In [19]:
df_name = list(set(df['中文名稱']))
name_dict = {}
for name in df_name:
    if name not in name_dict.keys():
        name_dict[name] = []
    if len(df_code['code'][(name == df_code['name'])]) > 0:
        name_dict[name] = df_code['code'][(name == df_code['name'])].iloc[0]
df['code'] = df['中文名稱'].apply(lambda x: name_dict[x])

In [20]:
DB = pd.merge(df,TEJ,left_on = ['中文名稱','財報發布日'], right_on = ['名稱','財報發布日'])

In [21]:
DB['收盤價'] = DB.swifter.apply(lambda x: x['收盤價'] if x['收盤價'] != '-' else x['結算價'], axis = 1) # 若當日流動量不足,改用結算價替代
DB['收盤價'] = DB['收盤價'].astype(float)

Pandas Apply: 100%|██████████| 3534900/3534900 [00:26<00:00, 131909.32it/s]


In [22]:
# 定义一个函数来计算百分比变化 # gpt 好棒
def calculate_pct_change(group):
    group['Return'] = group['收盤價'].pct_change()
    return group

# 使用 groupby 和 apply 来应用这个函数
DB = DB.groupby(['契約', '到期月份(週別)']).apply(calculate_pct_change)

In [23]:
DB_tmp = DB[['契約', '到期月份(週別)', '交易日期', 'Return']]

In [24]:
DB_tmp['交易日期'] = pd.to_datetime(DB_tmp['交易日期'])

# 使用 pivot_table 来重塑 DataFrame
pivot_df = DB_tmp.pivot_table(index='交易日期', columns=['契約', '到期月份(週別)'], values='Return')


/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
pivot_df.to_pickle("pivot_df.pkl")

In [45]:
tmp_df  = pd.read_pickle('pivot_df.pkl')
test_df = tmp_df.xs(201308, level='到期月份(週別)', axis = 1)

In [44]:
tmp_df

契約            CA1                     CAF                                    \
到期月份(週別)   201308 201309 201312    201101    201102    201103 201104 201105   
交易日期                                                                          
2011-01-04    NaN    NaN    NaN -0.002766 -0.002766 -0.002821    NaN    NaN   
2011-01-05    NaN    NaN    NaN -0.012483 -0.027739 -0.012730    NaN    NaN   
2011-01-06    NaN    NaN    NaN  0.005618  0.005706  0.005731    NaN    NaN   
2011-01-07    NaN    NaN    NaN -0.008380 -0.008511 -0.008547    NaN    NaN   
2011-01-10    NaN    NaN    NaN  0.002817  0.002861  0.002874    NaN    NaN   
...           ...    ...    ...       ...       ...       ...    ...    ...   
2022-12-26    NaN    NaN    NaN       NaN       NaN       NaN    NaN    NaN   
2022-12-27    NaN    NaN    NaN       NaN       NaN       NaN    NaN    NaN   
2022-12-28    NaN    NaN    NaN       NaN       NaN       NaN    NaN    NaN   
2022-12-29    NaN    NaN    NaN       NaN       NaN       NaN    NaN    NaN   
2022-12-30    NaN    NaN    NaN       NaN       NaN       NaN    NaN    NaN   

契約                           ...       RAF                        RBF         \
到期月份(週別)      201106 201107  ...    202303    202306    202309 202211 202212   
交易日期                         ...                                               
2011-01-04 -0.002762    NaN  ...       NaN       NaN       NaN    NaN    NaN   
2011-01-05 -0.012465    NaN  ...       NaN       NaN       NaN    NaN    NaN   
2011-01-06  0.005610    NaN  ...       NaN       NaN       NaN    NaN    NaN   
2011-01-07 -0.008368    NaN  ...       NaN       NaN       NaN    NaN    NaN   
2011-01-10  0.002813    NaN  ...       NaN       NaN       NaN    NaN    NaN   
...              ...    ...  ...       ...       ...       ...    ...    ...   
2022-12-26       NaN    NaN  ...  0.000000  0.000000  0.000000    NaN    NaN   
2022-12-27       NaN    NaN  ...  0.008969  0.008969  0.008969    NaN    NaN   
2022-12-28       NaN    NaN  ... -0.031111 -0.031111 -0.031111    NaN    NaN   
2022-12-29       NaN    NaN  ...  0.004587  0.004587  0.004587    NaN    NaN   
2022-12-30       NaN    NaN  ...  0.022831  0.022831  0.022831    NaN    NaN   

契約                                                            
到期月份(週別)      202301    202302    202303    202306    202309  
交易日期                                                          
2011-01-04       NaN       NaN       NaN       NaN       NaN  
2011-01-05       NaN       NaN       NaN       NaN       NaN  
2011-01-06       NaN       NaN       NaN       NaN       NaN  
2011-01-07       NaN       NaN       NaN       NaN       NaN  
2011-01-10       NaN       NaN       NaN       NaN       NaN  
...              ...       ...       ...       ...       ...  
2022-12-26  0.000000  0.000000  0.000000  0.000000  0.000000  
2022-12-27  0.006452  0.006460  0.006435  0.006443  0.006443  
2022-12-28 -0.019231 -0.019255 -0.019182 -0.019206 -0.019206  
2022-12-29 -0.007843 -0.007853 -0.007823 -0.007833 -0.007833  
2022-12-30  0.009223  0.009235  0.009198  0.009211  0.009211  

[2948 rows x 31491 columns]

# 找產業別, 組成Sector (可能採按市值加權)

In [27]:
#輸入有多少成分我們想要留住分解
pca = PCA(n_components=2)

#將資料分成兩個主成份
pca.fit(pivot_df)
x_pca = pca.transform(pivot_df)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# pivot_df.cumsum().plot() 

<AxesSubplot:xlabel='交易日期'>

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Glyph 20132 (\N{CJK UNIFIED IDEOGRAPH-4EA4}) missing from current font.
  func(*args, **kwargs)
/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Glyph 26131 (\N{CJK UNIFIED IDEOGRAPH-6613}) missing from current font.
  func(*args, **kwargs)
/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Glyph 26085 (\N{CJK UNIFIED IDEOGRAPH-65E5}) missing from current font.
  func(*args, **kwargs)
/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from current font.
  func(*args, **kwargs)
/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Glyph 22865 (\N{CJK UNIFIED IDEOGRAPH-5951}) missing from current font.
  func(*args, **kwargs)
/home/akil

ValueError: Image size of 546x659617 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 1 Axes>